### underthesea

In [2]:
from underthesea import ner

def predict_entities_underthesea(sentence):
    entities = ner(sentence)
    return entities

sentence = "Bộ Y tế yêu cầu TP HCM tổ chức khẩn cấp truy vết toàn bộ F1, F2 của bệnh nhân COVID-19 mới phát hiện."

entities = predict_entities_underthesea(sentence)
for entity in entities:
    print(f"Thực thể: {entity[0]}, Nhãn: {entity[3]}")


Thực thể: Bộ, Nhãn: O
Thực thể: Y tế, Nhãn: O
Thực thể: yêu cầu, Nhãn: O
Thực thể: TP, Nhãn: B-LOC
Thực thể: HCM, Nhãn: I-LOC
Thực thể: tổ chức, Nhãn: O
Thực thể: khẩn cấp, Nhãn: O
Thực thể: truy vết, Nhãn: O
Thực thể: toàn bộ, Nhãn: B-LOC
Thực thể: F1, Nhãn: I-LOC
Thực thể: ,, Nhãn: O
Thực thể: F2, Nhãn: I-LOC
Thực thể: của, Nhãn: O
Thực thể: bệnh nhân, Nhãn: B-LOC
Thực thể: COVID-19, Nhãn: I-LOC
Thực thể: mới, Nhãn: O
Thực thể: phát hiện, Nhãn: O
Thực thể: ., Nhãn: O


In [7]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from underthesea import ner

# Đọc file Parquet vào DataFrame
df = pd.read_parquet('0000.parquet')

# Chỉ lấy 200 dòng đầu tiên
df = df.head(200)

# Tiếp tục xử lý dữ liệu như thông thường
df['sentence'] = df['tokens'].apply(lambda tokens: ' '.join(tokens))
df['Ner'] = df['ner_tags'].apply(lambda tokens: ' '.join(tokens))

# Chuyển đổi dữ liệu
X = df['sentence'].tolist()
y = df['Ner'].tolist()

# Chia dữ liệu thành tập huấn luyện và tập kiểm tra theo tỷ lệ 8:2
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Dự đoán và thu thập kết quả với Underthesea
predicted_labels_underthesea = []
actual_labels_flat = []
accuracies_underthesea = []

for sentence, actual_labels in zip(X_test, y_test):
    predicted_words = ner(sentence)
    predicted_labels_sentence = [word[3] for word in predicted_words]
    
    # Chỉ lấy số lượng nhãn thực tế tương ứng với số lượng nhãn dự đoán để tránh lệch số lượng
    actual_labels_list = actual_labels.split()
    total_count = min(len(predicted_labels_sentence), len(actual_labels_list))
    
    correct_count = sum(predicted_label == actual_label for predicted_label, actual_label in zip(predicted_labels_sentence[:total_count], actual_labels_list[:total_count]))
    
    accuracy = correct_count / total_count if total_count > 0 else 0
    accuracies_underthesea.append(accuracy)
    
    actual_labels_flat.extend(actual_labels_list[:total_count])
    predicted_labels_underthesea.extend(predicted_labels_sentence[:total_count])

    # In ra kết quả cho từng câu
    print(f"Văn bản: {sentence}")
    print(f"Nhãn thực tế: {actual_labels_list[:total_count]}")
    print(f"Nhãn dự đoán: {predicted_labels_sentence[:total_count]}")
    print(f"Số từ dự đoán đúng: {correct_count} trên {total_count}")
    print(f"Tỷ lệ dự đoán đúng: {accuracy:.2%}")
    print()



Văn bản: Bệnh nhân hiện đã hết nCoV , điều trị tiếp tại Bệnh viện Chợ Rẫy .
Nhãn thực tế: ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-LOCATION']
Nhãn dự đoán: ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-LOC', 'I-LOC', 'O']
Số từ dự đoán đúng: 9 trên 12
Tỷ lệ dự đoán đúng: 75.00%

Văn bản: Những người vào trung tâm cách ly được xếp ở chung phòng một cách ngẫu nhiên .
Nhãn thực tế: ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
Nhãn dự đoán: ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
Số từ dự đoán đúng: 13 trên 13
Tỷ lệ dự đoán đúng: 100.00%

Văn bản: Bác sĩ Hùng cho biết diễn biến bệnh của bệnh nhân nhanh và phức tạp , khi vào viện đã trong tình trạng nguy kịch , phổi tổn thương nặng , suy hô hấp và đã có biến chứng suy đa tạng , tính mạng bị đe doạ .
Nhãn thực tế: ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-SYMPTOM_AND_DISEASE', 'I-SYMP

C:\Users\MSI\AppData\Roaming\Python\Python311\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\MSI\AppData\Roaming\Python\Python311\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\MSI\AppData\Roaming\Python\Python311\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result

In [8]:
# Tính trung bình cộng tỷ lệ dự đoán đúng mỗi câu với Underthesea
average_accuracy_underthesea = sum(accuracies_underthesea) / len(accuracies_underthesea) if accuracies_underthesea else 0
print(f"Tỷ lệ dự đoán đúng trung bình (Underthesea): {average_accuracy_underthesea:.2%}")

# In ra classification report cho Underthesea
print("\nBáo cáo phân loại (classification report) - Underthesea:")
print(classification_report(actual_labels_flat, predicted_labels_underthesea))

Tỷ lệ dự đoán đúng trung bình (Underthesea): 67.84%

Báo cáo phân loại (classification report) - Underthesea:
                       precision    recall  f1-score   support

                B-AGE       0.00      0.00      0.00         7
               B-DATE       0.00      0.00      0.00        15
             B-GENDER       0.00      0.00      0.00         5
                B-JOB       0.00      0.00      0.00         2
                B-LOC       0.00      0.00      0.00         0
           B-LOCATION       0.00      0.00      0.00        41
               B-MISC       0.00      0.00      0.00         0
               B-NAME       0.00      0.00      0.00         1
                B-ORG       0.00      0.00      0.00         0
       B-ORGANIZATION       0.00      0.00      0.00         5
         B-PATIENT_ID       0.00      0.00      0.00        27
                B-PER       0.00      0.00      0.00         0
B-SYMPTOM_AND_DISEASE       0.00      0.00      0.00         3
     B-

C:\Users\MSI\AppData\Roaming\Python\Python311\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\MSI\AppData\Roaming\Python\Python311\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\MSI\AppData\Roaming\Python\Python311\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result

## dữ liệu

In [6]:
import requests
API_URL = "https://datasets-server.huggingface.co/rows?dataset=truongpdd%2FNER-covid-vietnamese&config=default&split=train&offset=0&length=100"
def query():
    response = requests.get(API_URL)
    return response.json()
data = query()
data

{'features': [{'feature_idx': 0,
   'name': 'tokens',
   'type': {'feature': {'dtype': 'string', '_type': 'Value'},
    '_type': 'Sequence'}},
  {'feature_idx': 1,
   'name': 'ner_tags',
   'type': {'feature': {'dtype': 'string', '_type': 'Value'},
    '_type': 'Sequence'}}],
 'rows': [{'row_idx': 0,
   'row': {'tokens': ['Đồng',
     'thời',
     ',',
     'bệnh',
     'viện',
     'tiếp',
     'tục',
     'thực',
     'hiện',
     'các',
     'biện',
     'pháp',
     'phòng',
     'chống',
     'dịch',
     'bệnh',
     'COVID',
     '-',
     '19',
     'theo',
     'hướng',
     'dẫn',
     'của',
     'Bộ',
     'Y',
     'tế',
     '.'],
    'ner_tags': ['O',
     'O',
     'O',
     'O',
     'O',
     'O',
     'O',
     'O',
     'O',
     'O',
     'O',
     'O',
     'O',
     'O',
     'O',
     'O',
     'O',
     'O',
     'O',
     'O',
     'O',
     'O',
     'O',
     'B-ORGANIZATION',
     'I-ORGANIZATION',
     'I-ORGANIZATION',
     'O']},
   'truncated_cells': []

In [7]:
import requests
import pandas as pd

API_URL = "https://datasets-server.huggingface.co/rows?dataset=truongpdd%2FNER-covid-vietnamese&config=default&split=train&offset=0&length=100"

def query():
    response = requests.get(API_URL)
    return response.json()

data = query()

# Extracting tokens and ner_tags
rows = []
for row in data["rows"]:
    tokens = " ".join(row["row"]["tokens"])
    ner_tags = " ".join(row["row"]["ner_tags"])
    rows.append([tokens, ner_tags])

# Creating a DataFrame
df = pd.DataFrame(rows, columns=["tokens", "ner_tags"])

# Saving to an Excel file
file_path = "NER_covid_vietnamese.xlsx"
df.to_excel(file_path, index=False)

print(f"Data saved to {file_path}")


Data saved to NER_covid_vietnamese.xlsx


In [8]:
import requests
from sklearn.feature_extraction import DictVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

# Tải dữ liệu từ API
API_URL = "https://datasets-server.huggingface.co/rows?dataset=truongpdd%2FNER-covid-vietnamese&config=default&split=train&offset=0&length=100"
def query():
    response = requests.get(API_URL)
    return response.json()
data = query()

# Chuẩn bị dữ liệu huấn luyện
X_train = []
y_train = []

for row in data['rows']:
    tokens = row['row']['tokens']
    ner_tags = row['row']['ner_tags']
    for token, ner_tag in zip(tokens, ner_tags):
        X_train.append({'word': token})
        y_train.append(ner_tag)

# Biểu diễn các đặc trưng dưới dạng vector
vectorizer = DictVectorizer(sparse=False)
X_train_vectorized = vectorizer.fit_transform(X_train)

# Huấn luyện mô hình Random Forest
rf_model = RandomForestClassifier()
rf_model.fit(X_train_vectorized, y_train)

# Dữ liệu kiểm tra
X_test = [
    {'word': 'Apple'},
    {'word': 'là'},
    {'word': 'một'},
    {'word': 'công'},
    {'word': 'ty'},
    {'word': 'công'},
    {'word': 'nghệ'},
    {'word': 'lớn'},
    {'word': 'Nguyễn'},
    {'word': 'Văn'},
    {'word': 'A'},
    {'word': 'là'},
    {'word': 'CEO'},
    {'word': 'của'},
    {'word': 'công'},
    {'word': 'ty'},
    {'word': 'ABC'},
]

# Biểu diễn dữ liệu kiểm tra dưới dạng vector và dự đoán nhãn
X_test_vectorized = vectorizer.transform(X_test)
y_pred = rf_model.predict(X_test_vectorized)

# Hiển thị kết quả dự đoán
print("Kết quả dự đoán:")
for word, pred_label in zip(X_test, y_pred):
    print(f"{word['word']} - {pred_label}")

# Đánh giá mô hình
# (Chúng ta không có nhãn thật của tập kiểm tra nên không thể đánh giá ở đây)
# Nếu có nhãn thật, chúng ta có thể dùng classification_report
# y_test = [...] # Nhãn thật của X_test
# print(classification_report(y_test, y_pred))


Kết quả dự đoán:
Apple - O
là - O
một - O
công - O
ty - O
công - O
nghệ - O
lớn - O
Nguyễn - O
Văn - I-LOCATION
A - O
là - O
CEO - O
của - O
công - O
ty - O
ABC - O


In [9]:
import requests
from sklearn.feature_extraction import DictVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

# Tải dữ liệu từ API
API_URL = "https://datasets-server.huggingface.co/rows?dataset=truongpdd%2FNER-covid-vietnamese&config=default&split=train&offset=0&length=100"
def query():
    response = requests.get(API_URL)
    return response.json()
data = query()

# Chuẩn bị dữ liệu
X = []
y = []

for row in data['rows']:
    tokens = row['row']['tokens']
    ner_tags = row['row']['ner_tags']
    for token, ner_tag in zip(tokens, ner_tags):
        X.append({'word': token})
        y.append(ner_tag)

# Chia dữ liệu thành tập huấn luyện và tập kiểm tra theo tỷ lệ 8:2
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Biểu diễn các đặc trưng dưới dạng vector
vectorizer = DictVectorizer(sparse=False)
X_train_vectorized = vectorizer.fit_transform(X_train)
X_test_vectorized = vectorizer.transform(X_test)

# Huấn luyện mô hình Random Forest
rf_model = RandomForestClassifier()
rf_model.fit(X_train_vectorized, y_train)

# Dự đoán nhãn cho tập kiểm tra
y_pred = rf_model.predict(X_test_vectorized)
# Đánh giá mô hình
print(classification_report(y_test, y_pred))


                       precision    recall  f1-score   support

                B-AGE       0.00      0.00      0.00         2
               B-DATE       0.00      0.00      0.00         8
             B-GENDER       0.00      0.00      0.00         4
                B-JOB       0.00      0.00      0.00         1
           B-LOCATION       0.45      0.24      0.31        21
               B-NAME       1.00      0.50      0.67         2
       B-ORGANIZATION       1.00      0.67      0.80         6
         B-PATIENT_ID       0.20      0.09      0.12        11
B-SYMPTOM_AND_DISEASE       0.50      0.67      0.57         3
     B-TRANSPORTATION       0.00      0.00      0.00         2
               I-DATE       0.71      0.42      0.53        12
                I-JOB       0.00      0.00      0.00         3
           I-LOCATION       0.67      0.52      0.59        46
               I-NAME       0.00      0.00      0.00         0
       I-ORGANIZATION       0.29      0.11      0.16  

C:\Users\MSI\AppData\Roaming\Python\Python311\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\MSI\AppData\Roaming\Python\Python311\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\MSI\AppData\Roaming\Python\Python311\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result

In [11]:
from pyvi import ViTokenizer
# Hàm để dự đoán nhãn cho một câu
def predict_sentence(sentence):
    # tokens = ViTokenizer.tokenize(sentence).split()  # Tách câu thành các từ
    tokens = sentence.split()  # Tách câu thành các từ
    X_input = [{'word': token} for token in tokens]  # Chuẩn bị dữ liệu dưới dạng dictionary
    X_input_vectorized = vectorizer.transform(X_input)  # Biểu diễn các từ dưới dạng vector
    y_pred = rf_model.predict(X_input_vectorized)  # Dự đoán nhãn
    return list(zip(tokens, y_pred))

# Nhập câu từ người dùng
sentence = input("Nhập một câu: ")

# Dự đoán nhãn cho câu
predictions = predict_sentence(sentence)
for token, label in predictions:
    print(f"{token}: {label}")

Riêng: O
bệnh: O
nhân: O
91: B-PATIENT_ID
là: O
phi: B-JOB
công: O
người: O
Anh: O
ngụ: O
ở: O
quận: B-LOCATION
2: I-LOCATION
,: O
TP.: B-LOCATION
HCM: I-LOCATION
và: O
có: O
liên: O
quan: O
ổ: O
dịch: O
quán: O
bar: I-LOCATION
Buddha: I-LOCATION
,: O
thông: O
tin: O
cập: O
nhật: O
ngày: O
10: O
-: O
4: O
cho: O
biết: O
diễn: O
biến: O
bệnh: O
của: O
bệnh: O
nhân: O
không: O
xấu: O
hơn: O
nhưng: O
cũng: O
chưa: O
có: O
dấu: O
hiệu: O
hồi: O
phục: O
.: O
